In [3]:
import sagemaker
import boto3
from sagemaker.pytorch import PyTorch
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()
account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name
images_s3uri = 's3://{0}/lora/images/'.format(bucket)

In [19]:
##注意：lora train的图像目录下需要metadata.jsonl，其中file_name字段必须##########3
#imgs="https://d1xkebsgyt7kzd.cloudfront.net/R_1.jpg,https://d1xkebsgyt7kzd.cloudfront.net/R_2.jpg,https://d1xkebsgyt7kzd.cloudfront.net/R_3.jpg,https://d1xkebsgyt7kzd.cloudfront.net/R_4.jpg,https://d1xkebsgyt7kzd.cloudfront.net/R_5.jpg"
#for image in imgs.split(","):
#    !wget $image
#!mv ./*.jpg ./images/
!aws s3 cp images $images_s3uri --recursive

upload: images/.ipynb_checkpoints/dataset-checkpoint.toml to s3://sagemaker-ap-southeast-1-687912291502/dreambooth/images/.ipynb_checkpoints/dataset-checkpoint.toml
upload: images/00000-0-IMG20230302163943.txt to s3://sagemaker-ap-southeast-1-687912291502/dreambooth/images/00000-0-IMG20230302163943.txt
upload: images/00002-0-IMG20230302163946.txt to s3://sagemaker-ap-southeast-1-687912291502/dreambooth/images/00002-0-IMG20230302163946.txt
upload: images/00003-0-IMG20230302163947.txt to s3://sagemaker-ap-southeast-1-687912291502/dreambooth/images/00003-0-IMG20230302163947.txt
upload: images/dataset.toml to s3://sagemaker-ap-southeast-1-687912291502/dreambooth/images/dataset.toml
upload: images/00004-0-IMG20230302163948.txt to s3://sagemaker-ap-southeast-1-687912291502/dreambooth/images/00004-0-IMG20230302163948.txt
upload: images/00000-0-IMG20230302163943.png to s3://sagemaker-ap-southeast-1-687912291502/dreambooth/images/00000-0-IMG20230302163943.png
upload: images/00001-0-IMG202303021

In [20]:
#image_uri = '{0}.dkr.ecr.{1}.amazonaws.com/all-in-one-ai-stable-diffusion-webui-training'.format(account_id, region_name)
image_uri = '687912291502.dkr.ecr.ap-southeast-1.amazonaws.com/lora-finetuning:latest'
models_s3uri = 's3://{}/stable-diffusion/models/{}/{}'.format(bucket,'lora','model.tar.gz')

#################纯开源方式train############

In [24]:
import json
def json_encode_hyperparameters(hyperparameters):
    for (k, v) in hyperparameters.items():
        print(k, v)
    
    return {k: json.dumps(v) for (k, v) in hyperparameters.items()}

instance_type = 'ml.g4dn.xlarge'

s3_model_output_location='s3://{}/{}/{}'.format(bucket, 'lora', 'trained_models')
base_model_name="runwayml/stable-diffusion-v1-5"
output_dir="/opt/ml/model/"
#HUB_MODEL_ID="pokemon-lora"
dataset_name="lambdalabs/pokemon-blip-captions"

environment = {
    'PYTORCH_CUDA_ALLOC_CONF':'max_split_size_mb:32',
    'LD_LIBRARY_PATH':"${LD_LIBRARY_PATH}:/opt/conda/lib/"
}

hyperparameters = {
                    #'model_name':'aws-trained-lora-model',
                    'mixed_precision':'fp16',
                    'pretrained_model_name_or_path': base_model_name,
                    'dataset_name':dataset_name,
                    #'train_data_dir':'/opt/ml/input/data/images/',
                    'dataloader_num_workers':8,
                    'max_grad_norm':1,
                    'output_dir':output_dir,
                    'checkpointing_steps':1000,
                    'validation_prompt':'Totoro',
                    'seed':1338,
                    'manul_upload_model_path':s3_model_output_location,
                    'resolution':512,
                    'train_batch_size':1,
                    'gradient_accumulation_steps':4,
                    'learning_rate':2e-06,
                    'lr_scheduler':'cosine',
                    'lr_warmup_steps':0,
                    'max_train_steps':1000
}

hyperparameters = json_encode_hyperparameters(hyperparameters)

from sagemaker.estimator import Estimator
#inputs = {
#    'images': images_s3uri,
#    'models': models_s3uri
#}

estimator = Estimator(
    role = role,
    instance_count=1,
    instance_type = instance_type,
    image_uri = image_uri,
    hyperparameters = hyperparameters,
    environment = environment
)
estimator.fit()



mixed_precision fp16
pretrained_model_name_or_path runwayml/stable-diffusion-v1-5
dataset_name lambdalabs/pokemon-blip-captions
dataloader_num_workers 8
max_grad_norm 1
output_dir /opt/ml/model/
checkpointing_steps 1000
validation_prompt Totoro
seed 1338
manul_upload_model_path s3://sagemaker-ap-southeast-1-687912291502/lora/trained_models
resolution 512
train_batch_size 1
gradient_accumulation_steps 4
learning_rate 2e-06
lr_scheduler cosine
lr_warmup_steps 0
max_train_steps 1000
2023-03-06 14:44:56 Starting - Starting the training job...
2023-03-06 14:45:11 Starting - Preparing the instances for trainingProfilerReport-1678113896: InProgress
......
2023-03-06 14:46:18 Downloading - Downloading input data...
2023-03-06 14:46:38 Training - Downloading the training image.....................
2023-03-06 14:50:19 Training - Training image download completed. Training in progress..2023-03-06 14:50:20,468 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/

#################webui 插件方式train#######################

In [21]:
#!aws s3 rm --recursive $images_s3uri
!aws s3 ls $images_s3uri 

                           PRE .ipynb_checkpoints/
2023-03-16 02:01:06     262251 00000-0-IMG20230302163943.png
2023-03-16 02:01:06        144 00000-0-IMG20230302163943.txt
2023-03-16 02:01:06     269277 00001-0-IMG20230302163945.png
2023-03-16 02:01:06        116 00001-0-IMG20230302163945.txt
2023-03-16 02:01:06     287568 00002-0-IMG20230302163946.png
2023-03-16 02:01:06         95 00002-0-IMG20230302163946.txt
2023-03-16 02:01:06     289849 00003-0-IMG20230302163947.png
2023-03-16 02:01:06        166 00003-0-IMG20230302163947.txt
2023-03-16 02:01:06     275408 00004-0-IMG20230302163948.png
2023-03-16 02:01:06        189 00004-0-IMG20230302163948.txt
2023-03-16 02:01:06     267037 00005-0-IMG20230302163954.png
2023-03-16 02:01:06        169 00005-0-IMG20230302163954.txt
2023-03-16 02:01:06     296417 00006-0-IMG20230302163956.png
2023-03-16 02:01:06        115 00006-0-IMG20230302163956.txt
2023-03-16 02:01:06        270 dataset.toml


In [ ]:
import json
def json_encode_hyperparameters(hyperparameters):
    for (k, v) in hyperparameters.items():
        print(k, v)
    
    return {k: json.dumps(v) for (k, v) in hyperparameters.items()}

images_s3uri = 's3://{0}/dreambooth/images/'.format(bucket)
image_uri = '687912291502.dkr.ecr.ap-southeast-1.amazonaws.com/lora-finetuning-v2:latest'
base_model_name="runwayml/stable-diffusion-v1-5"
output_dir="/opt/ml/model/"
instance_type = 'ml.g4dn.xlarge'

environment = {
    'PYTORCH_CUDA_ALLOC_CONF':'max_split_size_mb:32',
    'LD_LIBRARY_PATH':"${LD_LIBRARY_PATH}:/opt/conda/lib/"
}

hyperparameters = {
                    'mixed_precision':'fp16',
                    'pretrained_model_name_or_path': base_model_name,
                    'dataset_config':'/opt/ml/input/data/images/dataset.toml',
                    'output_dir':output_dir,
                    'output_name':'aws-trained-lora-model',
                    'save_model_as':'safetensors', 
                    'prior_loss_weight':1.0, 
                    'max_train_steps':400, 
                    'learning_rate':1e-4, 
                    'optimizer_type':"AdamW8bit", 
                    'xformers':True, 
                    'mixed_precision':"fp16", 
                    'cache_latents':True, 
                    'gradient_checkpointing':True,
                    'save_every_n_epochs':100, 
                    'network_module':'networks.lora'
                    
}

hyperparameters = json_encode_hyperparameters(hyperparameters)

from sagemaker.estimator import Estimator
inputs = {
    'images': images_s3uri
}

estimator = Estimator(
    role = role,
    instance_count=1,
    instance_type = instance_type,
    image_uri = image_uri,
    hyperparameters = hyperparameters,
    environment = environment
)
estimator.fit(inputs)

mixed_precision fp16
pretrained_model_name_or_path runwayml/stable-diffusion-v1-5
dataset_config /opt/ml/input/data/images/dataset.toml
output_dir /opt/ml/model/
output_name aws-trained-lora-model
save_model_as safetensors
prior_loss_weight 1.0
max_train_steps 400
learning_rate 0.0001
optimizer_type AdamW8bit
xformers True
cache_latents True
gradient_checkpointing True
save_every_n_epochs 100
network_module networks.lora


INFO:sagemaker:Creating training-job with name: lora-finetuning-v2-2023-03-16-02-53-40-698


2023-03-16 02:53:41 Starting - Starting the training job...
2023-03-16 02:53:58 Starting - Preparing the instances for training......
2023-03-16 02:55:08 Downloading - Downloading input data
2023-03-16 02:55:08 Training - Downloading the training image...............

In [26]:
lora_model_data = estimator.model_data
print("Model artifact saved at:\n", lora_model_data)

Model artifact saved at:
 s3://sagemaker-ap-southeast-1-687912291502/lora-finetuning-2023-03-06-14-44-55-604/output/model.tar.gz


In [29]:
!aws s3 ls --human-readable --summarize s3://sagemaker-ap-southeast-1-687912291502/lora-finetuning-2023-03-06-14-44-55-604/output/model.tar.gz

2023-03-06 15:28:25   20.2 MiB model.tar.gz

Total Objects: 1
   Total Size: 20.2 MiB


In [40]:
!tar -xvf ./model.tar.gz

checkpoint-1000/
checkpoint-1000/pytorch_model.bin
checkpoint-1000/random_states_0.pkl
checkpoint-1000/optimizer.bin
checkpoint-1000/scheduler.bin
checkpoint-1000/scaler.pt
pytorch_lora_weights.bin
logs/
logs/text2image-fine-tune/
logs/text2image-fine-tune/1678114320.4189239/
logs/text2image-fine-tune/1678114320.4189239/events.out.tfevents.1678114320.ip-10-0-166-10.ap-southeast-1.compute.internal.23.1
logs/text2image-fine-tune/events.out.tfevents.1678114320.ip-10-0-166-10.ap-southeast-1.compute.internal.23.0
logs/text2image-fine-tune/1678114320.4203773/
logs/text2image-fine-tune/1678114320.4203773/hparams.yml
